# Yay Recommendations YAY

In [6]:
import pickle
from scipy.sparse import csr_matrix



In [2]:
with open ('combined_following_relationships.pkl', 'rb') as file_name:
    combined_df = pickle.load(file_name)
    file_name.close()

/Users/jennwon/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/jennwon/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [3]:
type(combined_df)

pandas.core.frame.DataFrame

In [4]:
combined_df.head()

,user,is_following,relationship
0,477797010,38611053,1
1,477797010,281433942,1
2,477797010,18088787,1
3,477797010,326424092,1
4,477797010,103880112,1


In [12]:
combined_df.user = combined_df.user.astype(int)
combined_df.is_following = combined_df.is_following.astype(int)
combined_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 9826739 entries, 0 to 3478499
Data columns (total 3 columns):
user            int64
is_following    int64
relationship    int64
dtypes: int64(3)
memory usage: 299.9 MB


In [13]:
sparse_matrix = csr_matrix((combined_df['relationship'], #this is the values that i care about
                            (combined_df['user'], combined_df['is_following']))) # these are the row/columns

In [22]:
# (sparse_matrix.shape) # (50.5116385, 50.5100818)
print(sparse_matrix)

  (171, 4391106)	1
  (189, 54450416)	1
  (196, 6228627)	1
  (235, 29063)	1
  (254, 20605616)	1
  (256, 41501527)	1
  (293, 4827016)	1
  (343, 40641)	1
  (343, 56627)	1
  (343, 910886)	1
  (357, 1159)	1
  (402, 290932)	1
  (410, 17512809)	1
  (425, 1758)	1
  (449, 7717052)	1
  (449, 45988850)	1
  (449, 52418005)	1
  (449, 160250093)	1
  (449, 163923460)	1
  (449, 171335681)	1
  (449, 214355273)	1
  (449, 219830046)	1
  (449, 250875231)	1
  (449, 255029545)	1
  (449, 271689246)	1
  :	:
  (504970227, 454240836)	1
  (504974829, 454240836)	1
  (504976440, 334655386)	1
  (504978252, 454240836)	1
  (504979395, 434444280)	1
  (504982170, 239723135)	1
  (504985047, 219577320)	1
  (504988794, 12090207)	1
  (504990651, 246617940)	1
  (504996744, 316875189)	1
  (505001196, 284152345)	1
  (505016706, 503566059)	1
  (505017075, 7840943)	1
  (505041669, 217850326)	1
  (505041669, 299523286)	1
  (505042206, 417146259)	1
  (505054344, 342381442)	1
  (505073523, 16148591)	1
  (505073523, 107172622)	1
  

# LMAO IM SO DUMB OK LETS START THE BETTER RECOMMENDER

In [11]:
# ok write a function that takes the user's likes
import requests
def get_user_likes(user_id):
    """
    this function takes a user_id and returns a list of all the songs that the user liked
    """
    client_id = 'kLve5Sh7JUSsPl5cCgQ7znRpV2fpzzu8'
    version = '1535697731'
    url = f'https://api-v2.soundcloud.com/users/{user_id}/likes?'
    #client_id={client_id}&limit=200&offset=0&linked_partitioning=1&app_version={version}&app_locale=en'
    parameters = {
        'client_id': 'kLve5Sh7JUSsPl5cCgQ7znRpV2fpzzu8',
        'limit' : 200,
        'offset' : 0,
        'linked_partition' : 1,
        'app_version': '1535697731',
        'app_locale' : 'en'
        
    }
    print(f'getting likes for {user_id}')
    try:
#         print(url)
        parsed = requests.get(url, params = parameters).json()
    except Exception as e:
        print(e)
    likes_list = []
    base_url = ''
    
    while parsed.get('next_href') != None: 
        # print('had next page')
        likes_list.extend(parsed.get('collection')) #adding what i already have
        offset = parsed.get('next_href').split('offset=')[1].split('&')[0]
        url = f'https://api-v2.soundcloud.com/users/{user_id}/likes?client_id={client_id}&limit=200&offset={offset}&linked_partitioning=1&app_version={version}&app_locale=en'
        parsed = requests.get(url).json()

    try :
        likes_list.extend(parsed.get('collection'))
        #if song.get('track', None) != None:
        
        songs_sans_id = [song.get('track', None) for song in likes_list]
#         print(f'length of songs before getting id: {len(songs_sans_id)}')
        likes_list = []
        for index, song in enumerate(songs_sans_id):
#             print(f'entered loop for index {index}')
            # print(song)
            if song != None:
                likes_list.append(song.get('id'))
        #songs_id = [song.get('id') for song in songs_sans_id if (song != None)]
        print(len(likes_list), '\n')
        # print((songs[0].keys()))
        #print(len(songs))
#         song_id = [song.get('id', None) for song in songs]
    except Exception as e:
        print(user_id)
        #print(likes_list)
        print(e)
        likes_list = []
    #print(len(likes_list))
    return likes_list #this is the collection list which includes all the users in the list_subject

In [12]:
# ok write a function that takes the user's likes
import requests
def get_user_streams(user_id):
    """
    this function takes a user_id and returns a list of all the songs that the user liked
    """
#     https://api-v2.soundcloud.com/stream/users/58899599?
# client_id=kLve5Sh7JUSsPl5cCgQ7znRpV2fpzzu8&limit=200&offset=0&linked_partitioning=1&app_version=1535697731&app_locale=en
    client_id = 'Dqvh7tFjir84G92sDsBuZXnNTOsTpZYa'
    version = '1536759207'
    url = f'https://api-v2.soundcloud.com/stream/users/{user_id}'
    #client_id={client_id}&limit=200&offset=0&linked_partitioning=1&app_version={version}&app_locale=en'
    parameters = {
        'client_id': 'kLve5Sh7JUSsPl5cCgQ7znRpV2fpzzu8',
        'limit' : 200,
        'offset' : 0,
        'linked_partition' : 1,
        'app_version': '1535697731',
        'app_locale' : 'en'
        
    }
    print(f'getting likes for {user_id}')
    try:
#         print(url)
        parsed = requests.get(url, params = parameters).json()
    except Exception as e:
        print(e)
    likes_list = []
    base_url = ''
    
    while parsed.get('next_href') != None: 
        # print('had next page')
        likes_list.extend(parsed.get('collection')) #adding what i already have
        offset = parsed.get('next_href').split('offset=')[1].split('&')[0]
        url = f'https://api-v2.soundcloud.com/users/{user_id}/likes?client_id={client_id}&limit=200&offset={offset}&linked_partitioning=1&app_version={version}&app_locale=en'
        parsed = requests.get(url).json()

    try :
        likes_list.extend(parsed.get('collection'))
        #if song.get('track', None) != None:
        
        songs_sans_id = [song.get('track', None) for song in likes_list]
#         print(f'length of songs before getting id: {len(songs_sans_id)}')
        likes_list = []
        for index, song in enumerate(songs_sans_id):
#             print(f'entered loop for index {index}')
            # print(song)
            if song != None:
                likes_list.append(song.get('id'))
        #songs_id = [song.get('id') for song in songs_sans_id if (song != None)]
        print(len(likes_list), '\n')
        # print((songs[0].keys()))
        #print(len(songs))
#         song_id = [song.get('id', None) for song in songs]
    except Exception as e:
        print(user_id)
        #print(likes_list)
        print(e)
        likes_list = []
    #print(len(likes_list))
    return likes_list #this is the collection list which includes all the users in the list_subject

In [13]:
user_id = 58899599
likes_id = get_user_likes(user_id)

getting likes for 58899599
182 



In [14]:
# ok now write a function that finds all the users linked to the user by max x degrees away

def get_following(user_tuple):
    user_id, degree = user_tuple
    print(f'I entered get_following, and I am at degree {degree} for user {user_id}', flush= True)
    client_id = 'Dqvh7tFjir84G92sDsBuZXnNTOsTpZYa'
    version = '1536759207'
    url = f'https://api-v2.soundcloud.com/users/{user_id}/followings?client_id={client_id}&limit=200&offset=0&linked_partitioning=1&app_version={version}&app_locale=en'
    try:
        parsed = requests.get(url).json()
    except Exception as e:
        print(e)
    following_list = []
    base_url = ''
    
    while parsed.get('next_href') != None: 
        following_list.extend(parsed.get('collection'))
        offset = parsed.get('next_href').split('offset=')[1].split('&')[0]
        url = f'https://api-v2.soundcloud.com/users/{user_id}/followings?client_id={client_id}&limit=200&offset={offset}&linked_partitioning=1&app_version={version}&app_locale=en'
        parsed = requests.get(url).json()

    following_list.extend(parsed.get('collection'))

    return [(song.get('id'), degree + 1) for song in following_list] #this is the collection list which includes all the users in the list_subject

In [15]:
following_id = get_following((58899599, 0)) #this returns a list of users that i follow and the degree from seed

I entered get_following, and I am at degree 0 for user 58899599


In [16]:
following_id

[(38611053, 1),
 (141186922, 1),
 (165051291, 1),
 (121978323, 1),
 (312963, 1),
 (3212849, 1),
 (27621001, 1),
 (169164060, 1),
 (27059681, 1),
 (214565468, 1),
 (119133497, 1),
 (145083453, 1),
 (120319217, 1),
 (4360546, 1),
 (3399796, 1),
 (52042226, 1),
 (18547356, 1),
 (2393812, 1),
 (8010696, 1),
 (33525051, 1),
 (488287, 1),
 (3157684, 1),
 (18542512, 1),
 (1555417, 1),
 (52154848, 1),
 (2947360, 1),
 (9152687, 1)]

In [21]:
from copy import deepcopy
import random
from collections import deque
# user_ids = deepcopy(Q)

Q = deque(random.sample(following_id, 2))
user_ids = deepcopy(Q)#i want 5 random users that I am following
degree_current = 1
while Q and degree_current < 3:
    new_user, degree = Q.popleft()  
    if degree < 3:
    # recall new_user is a tuple of user_id and degree from the seed
        following = get_following((new_user, degree)) # this fcn will print out the degree of the followings
        following = random.sample(following, min(len(following), 2))
        Q.extend(deque(following))
        user_ids.extend(following)
#         print(user_ids)
    degree_current = degree


I entered get_following, and I am at degree 1 for user 52154848
deque([(52154848, 1), (141186922, 1), (15838253, 2), (6136047, 2)])
I entered get_following, and I am at degree 1 for user 141186922
deque([(52154848, 1), (141186922, 1), (15838253, 2), (6136047, 2), (41671811, 2), (21457447, 2)])
I entered get_following, and I am at degree 2 for user 15838253
deque([(52154848, 1), (141186922, 1), (15838253, 2), (6136047, 2), (41671811, 2), (21457447, 2), (87231728, 3), (5140422, 3)])
I entered get_following, and I am at degree 2 for user 6136047
deque([(52154848, 1), (141186922, 1), (15838253, 2), (6136047, 2), (41671811, 2), (21457447, 2), (87231728, 3), (5140422, 3), (31727523, 3), (2098297, 3)])
I entered get_following, and I am at degree 2 for user 41671811
deque([(52154848, 1), (141186922, 1), (15838253, 2), (6136047, 2), (41671811, 2), (21457447, 2), (87231728, 3), (5140422, 3), (31727523, 3), (2098297, 3), (120987187, 3), (252907, 3)])
I entered get_following, and I am at degree 2 

In [22]:
len(user_ids)

14

In [23]:
from time import *
# ok now for these people get their likes
list_of_user_songs_dict = []
while user_ids:
    user_id, degree = user_ids.popleft()
    try:
        likes = get_user_likes(user_id)
        # print(len(likes))# returns a list of all the song_id that the user liked
        user_likes = {
        'user_id':user_id,
        'songs': likes,
        'degree':degree
        }
        list_of_user_songs_dict.append(user_likes)
#         sleep(2);
    except Exception as e:
        print(e)
    #break
        
    
    
    

getting likes for 52154848
123 

getting likes for 141186922
251 

getting likes for 15838253
81 

getting likes for 6136047
453 

getting likes for 41671811
0 

getting likes for 21457447
577 

getting likes for 87231728
1 

getting likes for 5140422
2 

getting likes for 31727523
23 

getting likes for 2098297
661 

getting likes for 120987187
4 

getting likes for 252907
6 

getting likes for 25968879
1 

getting likes for 7402148
13 



In [24]:
import numpy as np


In [25]:
# print(list_of_user_songs_dict[1])

{'user_id': 141186922, 'songs': [480159174, 182139857, 449204565, 160097478, 439037787, 422573118, 441596952, 428142006, 428810550, 428810619, 409054404, 393487224, 356057144, 382367201, 381253301, 363613115, 368643293, 371679506, 373130744, 330567729, 357804515, 233505497, 228501848, 176237086, 192408098, 344372146, 344525400, 347379577, 339318387, 345284640, 348867878, 139791623, 331261417, 330553971, 326097904, 332400509, 230795304, 179746710, 179746792, 179746888, 217031745, 339295603, 338504542, 331803840, 316845656, 331332422, 323874601, 328093943, 324820086, 191526274, 319678083, 320268000, 232764646, 321070937, 316964060, 317733357, 304822901, 294279693, 317010235, 314171510, 168712571, 312995844, 308246207, 306891184, 308066952, 305558912, 305157270, 303406208, 302391041, 303646929, 304154598, 302382926, 303599212, 231422683, 194198237, 262092630, 300680533, 298289587, 298712348, 296685652, 297753736, 297662583, 296419816, 265251355, 295949525, 260265693, 290163279, 293670200,

In [26]:
len(list_of_user_songs_dict)

14

Now I have to put the songs into a dataframe that has the song track and the degree and the user_id

In [40]:
import pandas as pd
all_songs = pd.DataFrame()
for user in list_of_user_songs_dict:
    user_df = pd.DataFrame(user)
    user_df['present'] = 1
    all_songs = all_songs.append(user_df)
#     print(user)
    #break

In [41]:
len(all_songs)

2196

In [51]:
all_songs.head()

all_songs.songs = all_songs.songs.astype(int)
all_songs.head()

,user_id,songs,degree,present
0,52154848,431218458,1,1
1,52154848,331341342,1,1
2,52154848,473543910,1,1
3,52154848,473555814,1,1
4,52154848,453371529,1,1


In [47]:
all_songs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2196 entries, 0 to 12
Data columns (total 4 columns):
user_id    2196 non-null int64
songs      2196 non-null int64
degree     2196 non-null int64
present    2196 non-null int64
dtypes: int64(4)
memory usage: 85.8 KB


In [56]:
import pandas as pd

# test_df = pd.DataFrame()
# test_df['songs'] = (list_of_user_songs_dict[0].get('songs'))
# test_df['user_id'] = list_of_user_songs_dict[0].get('user_id')
# test_df['present'] = 1

# test_df.head()

,songs,user_id,present
0,440627367,109859243,1
1,320550835,109859243,1
2,308704721,109859243,1
3,287673763,109859243,1
4,236934685,109859243,1


In [60]:
# ok now put all of them into a matrix so that i can do the recommendations
# testing.head()

,songs,user_id,present
0,494669505,284803,1
1,474527034,284803,1
2,248828138,284803,1
3,458716764,284803,1
4,260682299,284803,1


In [61]:
testing_2 = pd.DataFrame()
testing_2['songs'] = list_of_user_songs_dict[2].get('songs')
testing_2['user_id'] = list_of_user_songs_dict[2].get('user_id')
testing_2['present'] = 1

combined = pd.concat([test_df, testing_2], axis= 0)

len(combined)


114

In [62]:
# combined['user_id'].value_counts()

1231887      61
109859243    53
Name: user_id, dtype: int64

In [63]:
# testing.head()

,songs,user_id,present
0,494669505,284803,1
1,474527034,284803,1
2,248828138,284803,1
3,458716764,284803,1
4,260682299,284803,1


Doing SVD 

In [13]:
from sklearn.decomposition import TruncatedSVD
# from sklearn.random_projection import sparse_random_matrix

/Users/jennwon/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/jennwon/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [48]:
# wait, i dont think i even used this for the recommendation system wtf

from scipy.sparse import csr_matrix, csc_matrix

sparse_matrix = csc_matrix((all_songs['present'], #this is the values that i care about
                            (all_songs['user_id'],
                             all_songs['songs'])), dtype = float) # these are the row/columns

In [32]:
masks = {}
for hops in range(1,4):
    masks[hops] = np.array([d['user_id'] for d in list_of_user_songs_dict if d['degree'] == hops])

In [33]:
masks #this is a dictionary of the users that i am intersted in and their degree

{1: array([ 52154848, 141186922]),
 2: array([15838253,  6136047, 41671811, 21457447]),
 3: array([ 87231728,   5140422,  31727523,   2098297, 120987187,    252907,
         25968879,   7402148])}

In [50]:
masks[1]

array([ 52154848, 141186922])

In [53]:
# i iwould want to keep a dictionary that makes the song to the track id
# fix the q issue LOL

contrib = sum(sparse_matrix[masks[1], :])
# print(contrib)
for level in range(1,3): #this part needs to be fixed to include all the degress/hops away
    contrib += sum(sparse_matrix[masks[level + 1],:])/(level + 1) # should be +=

In [76]:
# np.argsort([10,5,1,3]) #this is a toy example

array([2, 3, 1, 0])

In [54]:
jenns_songs = np.argsort(contrib.toarray())

In [55]:
jenns_songs.reshape(-1,)[-10:][::-1]

array([449204565, 253413810, 216846955, 194771237, 261587226, 189525707,
       197311173, 179746710, 274807237, 217031745])

In [27]:
import pandas as pd
u = len((set(testing.user_id.values)))
p = len((set(testing.songs.values)))

sp_df = pd.DataFrame(sparse_matrix.todense(), index=u, columns=p)

MemoryError: 

In [16]:
import pickle

with open('test_sparse_matrix.pkl', 'wb') as file_name:
    pickle.dump(test_df, file_name)
    file_name.close()

In [7]:
import pickle

with open('test_sparse_matrix.pkl', 'rb') as file_name:
    testing = pickle.load(file_name)
    file_name.close()
    
print(testing[:5])

       songs  user_id  present
0  494669505   284803        1
1  474527034   284803        1
2  248828138   284803        1
3  458716764   284803        1
4  260682299   284803        1


In [3]:
#from scipy.sparse.coo_matrix import asfptype

#sparse_matrix.asfptype()

from scipy.sparse.linalg import svds
U, sigma, Vt = svds(sparse_matrix, k = 50)

KeyboardInterrupt: 